In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
973685,2021-02-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
973686,2021-02-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
973687,2021-02-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
973688,2021-02-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26559,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26561,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26563,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26565,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26567,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
604542,2021-02-06,Arkansas,Arkansas,5001,1870,37.00,5000,Arkansas,AR,Arkansas,State,3017804
604544,2021-02-07,Arkansas,Arkansas,5001,1881,37.00,5000,Arkansas,AR,Arkansas,State,3017804
604546,2021-02-08,Arkansas,Arkansas,5001,1884,37.00,5000,Arkansas,AR,Arkansas,State,3017804
604548,2021-02-09,Arkansas,Arkansas,5001,1894,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
973685,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
973686,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
973687,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
973688,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-10') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
386,2021-02-10,Snohomish,Washington,29556,512.00,53061,WA,County,822083,3595.26,62.28
770,2021-02-10,Cook,Illinois,462667,9601.00,17031,IL,County,5150233,8983.42,186.42
1153,2021-02-10,Orange,California,254417,3451.00,6059,CA,County,3175692,8011.39,108.67
1535,2021-02-10,Maricopa,Arizona,493720,8254.00,4013,AZ,County,4485414,11007.23,184.02
1917,2021-02-10,Los Angeles,California,1155821,18500.00,6037,CA,County,10039107,11513.19,184.28
...,...,...,...,...,...,...,...,...,...,...,...
973329,2021-02-10,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
973450,2021-02-10,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
973540,2021-02-10,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
973626,2021-02-10,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
386,2021-02-10,Snohomish,Washington,29556,512.00,53061,WA,County,822083,3595.26,62.28,62.28,3595.26
770,2021-02-10,Cook,Illinois,462667,9601.00,17031,IL,County,5150233,8983.42,186.42,186.42,8983.42
1153,2021-02-10,Orange,California,254417,3451.00,6059,CA,County,3175692,8011.39,108.67,108.67,8011.39
1535,2021-02-10,Maricopa,Arizona,493720,8254.00,4013,AZ,County,4485414,11007.23,184.02,184.02,11007.23
1917,2021-02-10,Los Angeles,California,1155821,18500.00,6037,CA,County,10039107,11513.19,184.28,184.28,11513.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
973329,2021-02-10,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
973450,2021-02-10,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
973540,2021-02-10,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
973626,2021-02-10,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
687551,2021-02-10,Clarke,Alabama,3352,42.00,1025,AL,County,23622,14190.16,177.80,177.80,14190.16,1
631981,2021-02-10,Hale,Alabama,2008,56.00,1065,AL,County,14651,13705.55,382.23,382.23,13705.55,2
495081,2021-02-10,Lowndes,Alabama,1282,42.00,1085,AL,County,9726,13181.16,431.83,431.83,13181.16,3
439208,2021-02-10,Etowah,Alabama,12861,284.00,1055,AL,County,102268,12575.78,277.70,277.70,12575.78,4
368826,2021-02-10,Franklin,Alabama,3929,74.00,1059,AL,County,31362,12527.90,235.95,235.95,12527.90,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603273,2021-02-10,Converse,Wyoming,932,15.00,56009,WY,County,13822,6742.87,108.52,108.52,6742.87,19
926853,2021-02-10,Platte,Wyoming,561,11.00,56031,WY,County,8393,6684.14,131.06,131.06,6684.14,20
832764,2021-02-10,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
748978,2021-02-10,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
495081,2021-02-10,Lowndes,Alabama,1282,42.00,1085,AL,County,9726,13181.16,431.83,431.83,13181.16,3,1
437916,2021-02-10,Clay,Alabama,1372,52.00,1027,AL,County,13235,10366.45,392.90,392.90,10366.45,24,2
631981,2021-02-10,Hale,Alabama,2008,56.00,1065,AL,County,14651,13705.55,382.23,382.23,13705.55,2,3
533718,2021-02-10,Greene,Alabama,835,31.00,1063,AL,County,8111,10294.66,382.20,382.20,10294.66,25,4
494437,2021-02-10,Crenshaw,Alabama,1372,50.00,1041,AL,County,13772,9962.24,363.06,363.06,9962.24,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532433,2021-02-10,Hot Springs,Wyoming,357,3.00,56017,WY,County,4413,8089.73,67.98,67.98,8089.73,15,19
730452,2021-02-10,Uinta,Wyoming,1995,12.00,56041,WY,County,20226,9863.54,59.33,59.33,9863.54,6,20
748978,2021-02-10,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22,21
210272,2021-02-10,Teton,Wyoming,3249,8.00,56039,WY,County,23464,13846.74,34.09,34.09,13846.74,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3595.26,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3595.26,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3595.26,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3595.26,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3595.26,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971382,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
971383,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
971384,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
971385,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
684933,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,177.80,14190.16,51,1,2.00
684934,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,177.80,14190.16,51,1,0.00
684935,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,177.80,14190.16,51,1,1.00
684936,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,177.80,14190.16,51,1,4.00
684937,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,177.80,14190.16,51,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761382,2021-02-06,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
761383,2021-02-07,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
761384,2021-02-08,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
761385,2021-02-09,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
493097,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13181.16,1,3,1.00,0.00
493098,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13181.16,1,3,0.00,0.00
493099,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13181.16,1,3,0.00,0.00
493100,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13181.16,1,3,0.00,0.00
493101,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13181.16,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491482,2021-02-06,Albany,Wyoming,3823,11.00,56001,WY,County,38880,9832.82,28.29,28.29,9871.40,23,5,5.00,0.00
491483,2021-02-07,Albany,Wyoming,3825,11.00,56001,WY,County,38880,9837.96,28.29,28.29,9871.40,23,5,2.00,0.00
491484,2021-02-08,Albany,Wyoming,3831,11.00,56001,WY,County,38880,9853.40,28.29,28.29,9871.40,23,5,6.00,0.00
491485,2021-02-09,Albany,Wyoming,3832,11.00,56001,WY,County,38880,9855.97,28.29,28.29,9871.40,23,5,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-10') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
252023,2021-02-10,Imperial,California,26549,573.00,6025,CA,County,181215,14650.55,316.20,316.20,14650.55,1,2,23.00,1.00
440452,2021-02-10,Inyo,California,1137,34.00,6027,CA,County,18039,6303.01,188.48,188.48,6303.01,2,32,11.00,2.00
1917,2021-02-10,Los Angeles,California,1155821,18500.00,6037,CA,County,10039107,11513.19,184.28,184.28,11513.19,3,5,3081.00,140.00
56405,2021-02-10,Stanislaus,California,53388,881.00,6099,CA,County,550660,9695.27,159.99,159.99,9695.27,4,10,110.00,7.00
69882,2021-02-10,Tulare,California,45704,661.00,6107,CA,County,466195,9803.62,141.79,141.79,9803.62,5,9,87.00,10.00
26217,2021-02-10,Riverside,California,284168,3454.00,6065,CA,County,2470546,11502.23,139.81,139.81,11502.23,6,6,643.00,40.00
399321,2021-02-10,Merced,California,27562,373.00,6047,CA,County,277680,9925.81,134.33,134.33,9925.81,7,8,53.00,6.00
533660,2021-02-10,Kings,California,21360,200.00,6031,CA,County,152940,13966.26,130.77,130.77,13966.26,8,3,18.00,1.00
25535,2021-02-10,Fresno,California,91730,1258.00,6019,CA,County,999101,9181.25,125.91,125.91,9181.25,9,14,300.00,4.00
44915,2021-02-10,San Joaquin,California,64457,943.00,6077,CA,County,762148,8457.28,123.73,123.73,8457.28,10,18,61.00,14.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
926404,2021-02-10,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,6.00,1.00
252023,2021-02-10,Imperial,California,26549,573.00,6025,CA,County,181215,14650.55,316.20,316.20,14650.55,1,2,23.00,1.00
533660,2021-02-10,Kings,California,21360,200.00,6031,CA,County,152940,13966.26,130.77,130.77,13966.26,8,3,18.00,1.00
125803,2021-02-10,San Bernardino,California,279673,2184.00,6071,CA,County,2180085,12828.54,100.18,100.18,12828.54,13,4,261.00,42.00
1917,2021-02-10,Los Angeles,California,1155821,18500.00,6037,CA,County,10039107,11513.19,184.28,184.28,11513.19,3,5,3081.00,140.00
26217,2021-02-10,Riverside,California,284168,3454.00,6065,CA,County,2470546,11502.23,139.81,139.81,11502.23,6,6,643.00,40.00
158049,2021-02-10,Kern,California,98636,664.00,6029,CA,County,900202,10957.10,73.76,73.76,10957.10,27,7,449.00,4.00
399321,2021-02-10,Merced,California,27562,373.00,6047,CA,County,277680,9925.81,134.33,134.33,9925.81,7,8,53.00,6.00
69882,2021-02-10,Tulare,California,45704,661.00,6107,CA,County,466195,9803.62,141.79,141.79,9803.62,5,9,87.00,10.00
56405,2021-02-10,Stanislaus,California,53388,881.00,6099,CA,County,550660,9695.27,159.99,159.99,9695.27,4,10,110.00,7.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
252023,2021-02-10,Imperial,California,26549,573.00,6025,CA,County,181215,14650.55,316.20,316.20,14650.55,1,2,23.00,1.00
440452,2021-02-10,Inyo,California,1137,34.00,6027,CA,County,18039,6303.01,188.48,188.48,6303.01,2,32,11.00,2.00
1917,2021-02-10,Los Angeles,California,1155821,18500.00,6037,CA,County,10039107,11513.19,184.28,184.28,11513.19,3,5,3081.00,140.00
56405,2021-02-10,Stanislaus,California,53388,881.00,6099,CA,County,550660,9695.27,159.99,159.99,9695.27,4,10,110.00,7.00
69882,2021-02-10,Tulare,California,45704,661.00,6107,CA,County,466195,9803.62,141.79,141.79,9803.62,5,9,87.00,10.00
26217,2021-02-10,Riverside,California,284168,3454.00,6065,CA,County,2470546,11502.23,139.81,139.81,11502.23,6,6,643.00,40.00
399321,2021-02-10,Merced,California,27562,373.00,6047,CA,County,277680,9925.81,134.33,134.33,9925.81,7,8,53.00,6.00
533660,2021-02-10,Kings,California,21360,200.00,6031,CA,County,152940,13966.26,130.77,130.77,13966.26,8,3,18.00,1.00
25535,2021-02-10,Fresno,California,91730,1258.00,6019,CA,County,999101,9181.25,125.91,125.91,9181.25,9,14,300.00,4.00
44915,2021-02-10,San Joaquin,California,64457,943.00,6077,CA,County,762148,8457.28,123.73,123.73,8457.28,10,18,61.00,14.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,316.20,14650.55,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,02/06/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,62.15,17983.19,34,1,0.00,1.00
3293,02/07/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,62.15,17983.19,34,1,0.00,0.00
3294,02/08/21,Lassen,California,5489,18.00,6035,CA,County,30573,17953.75,58.88,62.15,17983.19,34,1,54.00,0.00
3295,02/09/21,Lassen,California,5492,18.00,6035,CA,County,30573,17963.56,58.88,62.15,17983.19,34,1,3.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,316.20,14650.55,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,316.20,14650.55,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3366,02/06/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,123.73,8457.28,10,18,0.00,0.00
3367,02/07/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,123.73,8457.28,10,18,0.00,0.00
3368,02/08/21,San Joaquin,California,64138,923.00,6077,CA,County,762148,8415.43,121.11,123.73,8457.28,10,18,519.00,30.00
3369,02/09/21,San Joaquin,California,64396,929.00,6077,CA,County,762148,8449.28,121.89,123.73,8457.28,10,18,258.00,6.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)